In [ ]:
!pip install tensorflow pandas Pillow gradio numpy


In [199]:
import pandas as pd
kfooddb = pd.read_csv('food.csv', encoding='utf-8')

In [200]:
# 기존의 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models


In [201]:
# 데이터 로딩 및 제너레이터 설정 (기존과 동일)
data_dir = '/content/foodImage'
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],        # 밝기 범위 설정
    channel_shift_range=50,             # 채널(색상) 변형 설정
    validation_split=0.2
)


In [202]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=123
)

Found 2102 images belonging to 12 classes.


In [203]:
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=123
)

Found 520 images belonging to 12 classes.


In [204]:

# ---
# 3. 라벨 불일치 수정
# ImageDataGenerator가 찾은 클래스 이름을 직접 사용합니다.
class_names = list(train_generator.class_indices.keys())
print("모델이 인식한 음식 라벨:", class_names)

모델이 인식한 음식 라벨: ['.ipynb_checkpoints', '고사리나물', '곱창전골', '김치전', '깍두기', '달걀국', '달걀말이', '돼지갈비', '양념치킨', '전복죽', '호박전', '훈제오리']


In [205]:

# 1. 훈련 가능한 레이어 수 조정 (핵심 변경 부분)
# MobileNetV2 기반 모델 구축
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)


In [206]:
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-45:]:
    layer.trainable = True


In [207]:
# 새로운 분류 레이어 추가 (기존과 동일)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [208]:
class_weights = class_weight.compute_class_weight(
    "balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))


In [209]:
# ---
# 2. EarlyStopping 콜백 설정 (핵심 변경 부분)
callbacks = [
    # patience를 3으로 줄여 모델이 빠르게 과적합되기 전에 훈련을 중단
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    # ReduceLROnPlateau 설정은 유지
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
]


In [210]:
print("모델이 인식한 음식 라벨:", class_names)


모델이 인식한 음식 라벨: ['.ipynb_checkpoints', '고사리나물', '곱창전골', '김치전', '깍두기', '달걀국', '달걀말이', '돼지갈비', '양념치킨', '전복죽', '호박전', '훈제오리']


In [211]:
# 모델 컴파일 및 학습
model.compile(
    optimizer = Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [212]:
# model.fit() 함수에서 epochs 값을 변경하세요.

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=class_weights
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.2591 - loss: 2.3132 - val_accuracy: 0.5115 - val_loss: 1.4994 - learning_rate: 1.0000e-04
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.6853 - loss: 0.9442 - val_accuracy: 0.5885 - val_loss: 1.4159 - learning_rate: 1.0000e-04
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.8075 - loss: 0.6154 - val_accuracy: 0.6135 - val_loss: 1.3792 - learning_rate: 1.0000e-04
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.8315 - loss: 0.5137 - val_accuracy: 0.6096 - val_loss: 1.4036 - learning_rate: 1.0000e-04
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.8703 - loss: 0.4098 - val_accuracy: 0.6269 - val_loss: 1.2964 - learning_rate: 1.0000e-04
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.8692 - loss: 0.3939 - val_accuracy: 0.6808 - val_loss: 1.1575 - learning_rate: 1.0000e-04
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.9086 

In [213]:
# ---
# 3. 라벨 불일치 수정
# ImageDataGenerator가 찾은 클래스 이름을 직접 사용
class_names = list(train_generator.class_indices.keys())
print("모델이 인식한 음식 라벨:", class_names)

모델이 인식한 음식 라벨: ['.ipynb_checkpoints', '고사리나물', '곱창전골', '김치전', '깍두기', '달걀국', '달걀말이', '돼지갈비', '양념치킨', '전복죽', '호박전', '훈제오리']


In [214]:
# history 객체에서 정확도와 손실 값 확인
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
print("Training Accuracy:", train_accuracy[-1])
print("Validation Accuracy:", val_accuracy[-1])

Training Accuracy: 0.9690770506858826
Validation Accuracy: 0.7980769276618958


In [215]:
model.save('foodim9.keras')

In [ ]:
# 영양 DB 로드 (기존과 동일)
nutrition_db = pd.read_csv('/content/food.csv', encoding='UTF-8')
model = tf.keras.models.load_model("foodim6.keras")

In [ ]:
# Gradio 함수 수정
def predict_and_analyze_nutrition(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_food_name = class_names[predicted_class_index]

    nutrition_info = nutrition_db[nutrition_db['음 식 명'] == predicted_food_name]
    if not nutrition_info.empty:
        info = nutrition_info.iloc[0]
        result_text = (
            f"**분석 결과: {info['음 식 명']}**\n"
            f"--- (100g 기준 영양 정보) ---\n"
            f"칼로리: {info['에너지(kcal)']} kcal\n"
            f"단백질: {info['단백질(g)']} g\n"
            f"지방: {info['지방(g)']} g\n"
            f"탄수화물: {info['탄수화물(g)']} g\n"
        )
        return result_text
    else:
        return f"'{predicted_food_name}'에 대한 영양 정보를 찾을 수 없습니다."

In [ ]:
print(nutrition_db.columns)


Index(['음 식 명', '중량(g)', '에너지(kcal)', '탄수화물(g)', '당류(g)', '지방(g)', '단백질(g)',
       '칼슘(mg)', '인(mg)', '나트륨(mg)', '칼륨(mg)', '마그네슘(mg)', '철(mg)', '아연(mg)',
       '콜레스테롤(mg)', '트랜스지방(g)'],
      dtype='object')


In [ ]:
model.save('foodim4.keras')

In [165]:
model = tf.keras.models.load_model("foodim7.keras")

In [167]:
# history 객체에서 정확도와 손실 값 확인
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [168]:
print("Training Accuracy:", train_accuracy[-1])
print("Validation Accuracy:", val_accuracy[-1])

Training Accuracy: 0.9158512949943542
Validation Accuracy: 0.6480000019073486


In [169]:
# Load the trained model
trained_model = tf.keras.models.load_model('foodim7.keras')

In [ ]:
# 4. 모델 예측 수행
results = trained_model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
print("food_labels 길이:", len(food_labels))

food_labels 길이: 5


In [ ]:
print(data_dir)

/content/foodImage


In [ ]:
# Now the predict function should work correctly.
results = trained_model.predict(img_array)
print(results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
[[2.2989003e-07 1.6631458e-09 7.6916912e-08 1.9559285e-11 3.2113963e-03
  9.9678826e-01]]


In [216]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [217]:
# 1. 학습된 모델과 영양 DB 다시 로드
loaded_model = load_model('/content/foodim9.keras')
nutrition_db = pd.read_csv('/content/food.csv', encoding='UTF-8')

In [218]:
# 2. 클래스 인덱스-이름 매핑
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}

In [219]:
def predict_and_analyze_nutrition(img_path):
    # 이미지 전처리
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # 음식 예측
    predictions = loaded_model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_food_name = class_names[predicted_class_index]

    # 영양 정보 찾기
    nutrition_info = nutrition_db[nutrition_db['음 식 명'] == predicted_food_name]

    if not nutrition_info.empty:
        info = nutrition_info.iloc[0]
        result_text = (
            f"**분석 결과: {info['음 식 명']}**\n"
            f"--- (100g 기준 영양 정보) ---\n"
            f"칼로리: {info['에너지(kcal)']} kcal\n"
            f"단백질: {info['단백질(g)']} g\n"
            f"지방: {info['지방(g)']} g\n"
            f"탄수화물: {info['탄수화물(g)']} g\n"
        )
        return result_text
    else:
        return f"'{predicted_food_name}'에 대한 영양 정보를 찾을 수 없습니다."

In [220]:
# 3. Gradio 인터페이스로 사용자에게 보여주기
import gradio as gr

iface = gr.Interface(
    fn=predict_and_analyze_nutrition,
    inputs=gr.Image(type="filepath", label="음식 이미지 업로드"),
    outputs=gr.Textbox(label="영양 분석 결과"),
    title="음식 영양 분석기 🥗",
    description="음식 이미지를 업로드하면 영양소 정보를 예측하고 분석합니다."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65c527ef29c65a5ef5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
